In [39]:
import pandas as pd
import numpy as np

In [40]:
routes = pd.read_csv("All_Data/routes.txt")
trips = pd.read_csv("All_Data/trips.txt")
stops = pd.read_csv("All_Data/stops.txt")
stop_times = pd.read_csv("All_Data/stop_times.txt")

In [41]:
routes.head()

,agency_id,route_id,route_long_name,route_short_name,route_type
0,DIMTS,142,828AUP,NaN,3
1,DIMTS,10001,971DOWN,NaN,3
2,DIMTS,362,961DOWN,NaN,3
3,DIMTS,274,819DOWN,NaN,3
4,DIMTS,10386,824STLDOWN2,NaN,3


In [42]:
trips.head()

,route_id,service_id,trip_id,shape_id
0,142,1,142_12_50,NaN
1,142,1,142_11_17,NaN
2,142,1,142_09_14,NaN
3,142,1,142_14_00,NaN
4,142,1,142_10_44,NaN


In [43]:
unique_trips = trips.groupby('route_id').first().reset_index()
unique_trips.head()

,route_id,service_id,trip_id,shape_id
0,1,1,1_23_16,NaN
1,2,1,2_12_10,NaN
2,4,1,4_13_13,NaN
3,6,1,6_14_08,NaN
4,8,1,8_07_45,NaN


In [44]:
routes_trips_merge = pd.merge(routes, trips, on='route_id')

In [45]:
routes_trips_merge

,agency_id,route_id,route_long_name,route_short_name,route_type,service_id,trip_id,shape_id
0,DIMTS,142,828AUP,NaN,3,1,142_12_50,NaN
1,DIMTS,142,828AUP,NaN,3,1,142_11_17,NaN
2,DIMTS,142,828AUP,NaN,3,1,142_09_14,NaN
3,DIMTS,142,828AUP,NaN,3,1,142_14_00,NaN
4,DIMTS,142,828AUP,NaN,3,1,142_10_44,NaN
...,...,...,...,...,...,...,...,...
89388,DTC,10670,244UP,NaN,3,1,10670_07_35,NaN
89389,DTC,10670,244UP,NaN,3,1,10670_15_45,NaN
89390,DTC,10670,244UP,NaN,3,1,10670_15_00,NaN
89391,DTC,10670,244UP,NaN,3,1,10670_07_16,NaN


In [46]:
trip_to_route_map = routes_trips_merge.set_index('trip_id')['route_id'].to_dict()

In [47]:
stop_times['trip_id'] = stop_times['trip_id'].map(trip_to_route_map)

In [48]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,142,12:50:00,12:50:00,146,0
1,142,12:51:11,12:51:11,148,1
2,142,12:52:49,12:52:49,149,2
3,142,12:53:54,12:53:54,488,3
4,142,12:55:44,12:55:44,233,4


In [49]:
stop_times.rename(columns={'trip_id': 'route_id'}, inplace=True)
stop_times.columns

Index(['route_id', 'arrival_time', 'departure_time', 'stop_id',
       'stop_sequence'],
      dtype='object')

In [50]:
stop_merge = pd.merge(stop_times, stops, on='stop_id')
stop_merge.head()

,route_id,arrival_time,departure_time,stop_id,stop_sequence,stop_code,stop_lat,stop_lon,stop_name,zone_id
0,142,12:50:00,12:50:00,146,0,U00191,28.614614,76.978024,Najafgarh Terminal,146
1,142,12:51:11,12:51:11,148,1,DC3555,28.614481,76.981062,Jharoda Crossing,148
2,142,12:52:49,12:52:49,149,2,DC3556,28.614150,76.985217,Health Centre,149
3,142,12:53:54,12:53:54,488,3,U00233,28.611746,76.985721,NajafGarh Delhi Gate,488
4,142,12:55:44,12:55:44,233,4,U00226,28.610319,76.981338,Police Station NajafGarh,233


In [51]:
stop_merge.to_csv('stop_merge.csv', index=False)

In [53]:
trip_count = pd.DataFrame(trips.groupby('route_id').size())
trip_count.rename(columns={0: 'trip_count'}, inplace=True)
print(trip_count.head())
trip_count['trip_count'] = round(trip_count['trip_count'] / 5) + 1
print(trip_count.head())
route_trip_count = pd.merge(routes, trip_count, on='route_id')
route_trip_count.head()
route_trip_count.sort_values('trip_count',ascending=False)

          trip_count
route_id            
1                 14
2                 22
4                 20
6                 92
8                 28
          trip_count
route_id            
1                4.0
2                5.0
4                5.0
6               19.0
8                7.0


,agency_id,route_id,route_long_name,route_short_name,route_type,trip_count
2121,DTC,5722,623ADOWN,NaN,3,65.0
2120,DTC,5721,623DOWN,NaN,3,65.0
596,DIMTS,674,628STLDOWN,NaN,3,64.0
757,DIMTS,593,628STLUP,NaN,3,63.0
1653,DTC,5254,623UP,NaN,3,55.0
...,...,...,...,...,...,...
1695,DTC,5296,770BUP,NaN,3,1.0
1696,DTC,5297,773UP,NaN,3,1.0
1643,DTC,5244,605UP,NaN,3,1.0
1630,DTC,5231,548UP,NaN,3,1.0


In [56]:
stop_times.head()

,route_id,arrival_time,departure_time,stop_id,stop_sequence
0,142,12:50:00,12:50:00,146,0
1,142,12:51:11,12:51:11,148,1
2,142,12:52:49,12:52:49,149,2
3,142,12:53:54,12:53:54,488,3
4,142,12:55:44,12:55:44,233,4


In [55]:
trips['start_time'] = stop_times.groupby('trip_id')['arrival_time'].first()

KeyError: 'trip_id'